# Hands-On: Strong and Weak Scaling with SLURM

**Objective:** Apply Amdahl's and Gustafson's Laws in practice by measuring strong and weak scaling on an HPC cluster.

In this exercise, you will:
1. Compile an OpenMP-parallelized Julia set computation
2. Submit SLURM job arrays to measure strong and weak scaling
3. Analyze the results and compare them to theoretical predictions

---

**Acknowledgment:** This exercise is inspired by the tutorial ["Scalability: Strong and Weak Scaling"](https://www.kth.se/blogs/pdc/2018/11/scalability-strong-and-weak-scaling/) from KTH PDC, which references the Julia set OpenMP code from [John Burkardt (FSU)](https://people.sc.fsu.edu/~jburkardt/c_src/julia_set_openmp/julia_set_openmp.c).

---

### Using Jupyter Notebooks

If you're new to Jupyter notebooks, here are the essentials:
- **Shift+Enter**: Execute the current cell and move to the next one
- **Ctrl+Enter**: Execute the current cell and stay on it
- Cells can contain **code** (Python) or **Markdown** (formatted text)
- Execute cells in order from top to bottom
- If something goes wrong, you can restart the kernel via *Kernel → Restart*

## 1. Theory Recap

### Strong Scaling (Amdahl's Law)

**Question:** How much faster can we solve a *fixed-size* problem by adding more processors?

$$S(N) = \frac{1}{(1-p) + \frac{p}{N}}$$

Where:
- $S(N)$ = speedup with $N$ processors
- $p$ = parallelizable fraction of the code
- $(1-p)$ = serial fraction

**Key insight:** Even with infinite processors, speedup is limited by the serial fraction: $S_{max} = \frac{1}{1-p}$

### Weak Scaling (Gustafson's Law)

**Question:** How much larger a problem can we solve in the *same time* by adding more processors?

$$S(N) = N - s(N-1) = s + p \cdot N$$

Where:
- $s$ = serial fraction
- $p = 1 - s$ = parallel fraction

**Key insight:** Speedup grows linearly with $N$ — more optimistic for large-scale computing!

## 2. The Julia Set Algorithm

The [Julia set](https://en.wikipedia.org/wiki/Julia_set) is a fractal defined by iterating the complex function:

$$Z_{k+1} = Z_k^2 + C$$

For each pixel (mapped to a complex number $Z_0$), we iterate until either:
- The magnitude $|Z_k|$ exceeds a threshold (point escapes → not in set)
- We reach the maximum iterations (point stays bounded → in set)

The iteration count determines the pixel color, producing beautiful fractal images.

**Why Julia set for scaling tests?**
- Each pixel is computed independently → embarrassingly parallel
- Computation time scales linearly with image size
- Easy to vary problem size (just change resolution)

We use $C = -0.8 + 0.156i$ which produces a visually interesting fractal.

## 3. Setup and Configuration

First, let's configure matplotlib and set experiment parameters.

In [ ]:
# Matplotlib configuration for interactive figures
%matplotlib inline

import matplotlib
matplotlib.rcParams['figure.figsize'] = (10, 8)
matplotlib.rcParams['figure.dpi'] = 100

In [ ]:
# =============================================================================
# EXPERIMENT PARAMETERS - Adjust these as needed
# =============================================================================

# Artificial serial time (in milliseconds) to add to each run.
# This makes the effect of Amdahl's Law more visible in the plots.
# Set to 0 for pure Julia set computation timing.
ARTIFICIAL_SERIAL_MS = 100  # Try different values: 0, 50, 100, 200

# Problem sizes
STRONG_SCALING_SIZE = 4000  # Fixed size for strong scaling (width = height)
WEAK_SCALING_BASE = 1000    # Base size for weak scaling (1 thread)

print(f"Artificial serial time: {ARTIFICIAL_SERIAL_MS} ms")
print(f"Strong scaling: {STRONG_SCALING_SIZE} x {STRONG_SCALING_SIZE} pixels")
print(f"Weak scaling base: {WEAK_SCALING_BASE} x {WEAK_SCALING_BASE} pixels")

In [ ]:
import os

# Create directories for logs and output
os.makedirs('logs', exist_ok=True)
os.makedirs('output', exist_ok=True)

print("Directories created:")
print("  - logs/   (SLURM output files)")
print("  - output/ (Julia set images)")

## 4. The C Code

The following cell writes the OpenMP-parallelized Julia set code to `julia_set.c`.

Key features:
- Uses `#pragma omp parallel for` to distribute pixel computation across threads
- Uses `omp_get_wtime()` for precise timing
- Accepts an optional **artificial serial delay** to demonstrate Amdahl's Law
- Outputs timing info to stdout (captured by SLURM)
- Writes image as binary file (easy to read in Python)

In [ ]:
%%writefile julia_set.c
/*
 * Julia Set computation with OpenMP parallelization
 * 
 * Inspired by the KTH PDC scaling tutorial and code from John Burkardt (FSU):
 * https://www.kth.se/blogs/pdc/2018/11/scalability-strong-and-weak-scaling/
 * https://people.sc.fsu.edu/~jburkardt/c_src/julia_set_openmp/julia_set_openmp.c
 *
 * Usage: ./julia_set <width> <height> <output_file> [serial_ms]
 *   width, height : image dimensions
 *   output_file   : binary output file
 *   serial_ms     : optional artificial serial delay in milliseconds (default: 0)
 *
 * Thread count is controlled via OMP_NUM_THREADS environment variable.
 */

#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include <time.h>
#include <omp.h>

/* Julia set parameters */
#define C_REAL -0.8
#define C_IMAG  0.156
#define MAX_ITER 200
#define ESCAPE_RADIUS 1000.0

/* Domain boundaries */
#define X_MIN -1.5
#define X_MAX  1.5
#define Y_MIN -1.5
#define Y_MAX  1.5

/*
 * Artificial serial delay using busy-wait.
 * This simulates a non-parallelizable portion of the code.
 */
void artificial_serial_delay(double milliseconds) {
    if (milliseconds <= 0) return;
    
    double seconds = milliseconds / 1000.0;
    double start = omp_get_wtime();
    
    /* Busy-wait loop */
    while (omp_get_wtime() - start < seconds) {
        /* Do nothing - just wait */
    }
}

/*
 * Compute the iteration count for a single point.
 * Returns the number of iterations before escape, or MAX_ITER if bounded.
 */
int julia_iterate(double x0, double y0) {
    double x = x0;
    double y = y0;
    int iter;
    
    for (iter = 0; iter < MAX_ITER; iter++) {
        double x_new = x * x - y * y + C_REAL;
        double y_new = 2.0 * x * y + C_IMAG;
        x = x_new;
        y = y_new;
        
        if (x * x + y * y > ESCAPE_RADIUS * ESCAPE_RADIUS) {
            break;
        }
    }
    
    return iter;
}

/*
 * Compute the Julia set for the entire image.
 * This is where OpenMP parallelization happens.
 */
void compute_julia_set(unsigned char *image, int width, int height) {
    double x_scale = (X_MAX - X_MIN) / (double)(width - 1);
    double y_scale = (Y_MAX - Y_MIN) / (double)(height - 1);
    
    #pragma omp parallel for schedule(dynamic)
    for (int j = 0; j < height; j++) {
        double y0 = Y_MAX - j * y_scale;  /* Flip y for image coordinates */
        
        for (int i = 0; i < width; i++) {
            double x0 = X_MIN + i * x_scale;
            int iter = julia_iterate(x0, y0);
            
            /* Map iteration count to grayscale (0-255) */
            image[j * width + i] = (unsigned char)(iter % 256);
        }
    }
}

/*
 * Write the image as a binary file.
 * Format: [width (4 bytes)] [height (4 bytes)] [pixels (width*height bytes)]
 */
int write_binary_image(const char *filename, unsigned char *image, int width, int height) {
    FILE *fp = fopen(filename, "wb");
    if (fp == NULL) {
        fprintf(stderr, "Error: Cannot open file %s for writing\n", filename);
        return -1;
    }
    
    /* Write header: width and height as 32-bit integers */
    int32_t w = width;
    int32_t h = height;
    fwrite(&w, sizeof(int32_t), 1, fp);
    fwrite(&h, sizeof(int32_t), 1, fp);
    
    /* Write pixel data */
    fwrite(image, sizeof(unsigned char), width * height, fp);
    
    fclose(fp);
    return 0;
}

int main(int argc, char *argv[]) {
    if (argc < 4) {
        printf("Usage: %s <width> <height> <output_file> [serial_ms]\n", argv[0]);
        printf("  serial_ms: artificial serial delay in milliseconds (default: 0)\n");
        printf("  Thread count is controlled via OMP_NUM_THREADS.\n");
        return 1;
    }
    
    int width = atoi(argv[1]);
    int height = atoi(argv[2]);
    const char *output_file = argv[3];
    double serial_ms = (argc > 4) ? atof(argv[4]) : 0.0;
    
    /* Get thread count */
    int num_threads;
    #pragma omp parallel
    {
        #pragma omp single
        num_threads = omp_get_num_threads();
    }
    
    /* Allocate image buffer */
    unsigned char *image = (unsigned char *)malloc(width * height * sizeof(unsigned char));
    if (image == NULL) {
        fprintf(stderr, "Error: Cannot allocate memory for %dx%d image\n", width, height);
        return 1;
    }
    
    /* Start timing */
    double start_time = omp_get_wtime();
    
    /* Artificial serial portion (not parallelizable) */
    artificial_serial_delay(serial_ms);
    
    /* Compute Julia set (parallel portion) */
    compute_julia_set(image, width, height);
    
    /* End timing */
    double end_time = omp_get_wtime();
    double elapsed = end_time - start_time;
    
    /* Output timing information (parsed by Python later) */
    printf("Threads: %d\n", num_threads);
    printf("Width: %d\n", width);
    printf("Height: %d\n", height);
    printf("SerialMS: %.1f\n", serial_ms);
    printf("Time: %.6f\n", elapsed);
    
    /* Write output image */
    if (write_binary_image(output_file, image, width, height) != 0) {
        free(image);
        return 1;
    }
    
    printf("Output: %s\n", output_file);
    
    free(image);
    return 0;
}

## 5. Compilation

Compile the code with OpenMP support. Run this command in your terminal:

```bash
gcc -O3 -fopenmp -o julia_set julia_set.c
```

Flags:
- `-O3`: Aggressive optimization
- `-fopenmp`: Enable OpenMP support

In [ ]:
# Compile from within the notebook:
!gcc -O3 -fopenmp -o julia_set julia_set.c && echo "Compilation successful!"

## 6. Local Test (Optional)

Before submitting to SLURM, let's verify the code works locally with a small test:

In [ ]:
# Quick local test with 2 threads, small image, and artificial serial delay
!OMP_NUM_THREADS=2 ./julia_set 500 500 output/test_local.bin {ARTIFICIAL_SERIAL_MS}

In [ ]:
# Visualize the test output
import numpy as np
import matplotlib.pyplot as plt

def read_julia_image(filename):
    """Read a binary Julia set image file."""
    with open(filename, 'rb') as f:
        # Read header
        width = np.frombuffer(f.read(4), dtype=np.int32)[0]
        height = np.frombuffer(f.read(4), dtype=np.int32)[0]
        # Read pixel data
        pixels = np.frombuffer(f.read(), dtype=np.uint8)
        return pixels.reshape(height, width), width, height

# Display the test image
img, w, h = read_julia_image('output/test_local.bin')
plt.figure(figsize=(8, 8))
plt.imshow(img, cmap='hot')
plt.colorbar(label='Iterations')
plt.title('Julia Set (local test)')
plt.axis('off')
plt.show()

---

## 7. Strong Scaling Experiment

**Goal:** Measure how execution time decreases as we add more cores to a *fixed-size* problem.

**Setup:**
- Fixed problem size: **4000 × 4000** pixels
- Artificial serial delay: configurable (default 100ms)
- Vary thread count: 1, 2, 4, 8, 16, 24, 32, 48
- Reserve full node (`--cpus-per-task 48`) to avoid interference from other jobs

**Expected behavior:** Speedup will increase with cores but eventually plateau (Amdahl's Law).

In [ ]:
# Generate the SLURM script with current parameters
strong_scaling_script = f'''#!/bin/bash -l

# =============================================================================
# Strong Scaling Experiment - Julia Set
# Fixed problem size ({STRONG_SCALING_SIZE}x{STRONG_SCALING_SIZE}), varying thread count
# =============================================================================

# Job general details
#SBATCH --job-name JuliaSet_StrongScaling
#SBATCH --account rfabbret_cours_hpc
#SBATCH --mail-type NONE
#SBATCH --time 00:29:59

# Paths and output
#SBATCH --output logs/strong_scaling_%A_%a.out

# Resources - reserve full node to avoid interference
#SBATCH --nodes 1
#SBATCH --ntasks 1
#SBATCH --cpus-per-task 48
#SBATCH --mem 8G

# Node specificities
#SBATCH --partition cpu

# Array: each task ID is the number of threads to use
#SBATCH --array 1,2,4,8,16,24,32,48

# Clean environment
#SBATCH --export NONE

# =============================================================================
# Job execution
# =============================================================================

# Set thread count from array task ID
export OMP_NUM_THREADS=${{SLURM_ARRAY_TASK_ID}}

# Fixed problem size for strong scaling
WIDTH={STRONG_SCALING_SIZE}
HEIGHT={STRONG_SCALING_SIZE}

# Artificial serial delay (ms)
SERIAL_MS={ARTIFICIAL_SERIAL_MS}

# Output file
OUTPUT_FILE="output/strong_${{SLURM_ARRAY_TASK_ID}}cores.bin"

echo "=== Strong Scaling Experiment ==="
echo "Job ID: ${{SLURM_JOB_ID}}"
echo "Array Task ID: ${{SLURM_ARRAY_TASK_ID}}"
echo "Threads: ${{OMP_NUM_THREADS}}"
echo "Problem size: ${{WIDTH}} x ${{HEIGHT}}"
echo "Artificial serial delay: ${{SERIAL_MS}} ms"
echo ""

# Run the Julia set computation
./julia_set ${{WIDTH}} ${{HEIGHT}} ${{OUTPUT_FILE}} ${{SERIAL_MS}}
'''

with open('job_strong_scaling.sh', 'w') as f:
    f.write(strong_scaling_script)

print("Generated job_strong_scaling.sh")
print(f"  - Problem size: {STRONG_SCALING_SIZE}x{STRONG_SCALING_SIZE}")
print(f"  - Artificial serial delay: {ARTIFICIAL_SERIAL_MS} ms")

### Submit the Strong Scaling Job

Run this command in your terminal:

```bash
sbatch job_strong_scaling.sh
```

Monitor your jobs with:
```bash
squeue -u $USER
```

Wait for all array tasks to complete before proceeding to the analysis.

---

## 8. Weak Scaling Experiment

**Goal:** Measure how execution time stays constant as we scale *both* problem size and cores proportionally.

**Setup:**
- Base problem size: **1000 × 1000** pixels for 1 core
- Scale both dimensions by $\sqrt{N}$ where $N$ is the thread count
- This keeps work per thread constant (each thread processes ~1M pixels)

| Threads | Width | Height | Total Pixels | Pixels/Thread |
|---------|-------|--------|--------------|---------------|
| 1 | 1000 | 1000 | 1M | 1M |
| 4 | 2000 | 2000 | 4M | 1M |
| 16 | 4000 | 4000 | 16M | 1M |
| 48 | 6928 | 6928 | ~48M | 1M |

**Expected behavior:** Execution time should remain roughly constant (Gustafson's Law).

In [ ]:
# Generate the SLURM script with current parameters
weak_scaling_script = f'''#!/bin/bash -l

# =============================================================================
# Weak Scaling Experiment - Julia Set
# Problem size scales with thread count (constant work per thread)
# =============================================================================

# Job general details
#SBATCH --job-name JuliaSet_WeakScaling
#SBATCH --account rfabbret_cours_hpc
#SBATCH --mail-type NONE
#SBATCH --time 00:29:59

# Paths and output
#SBATCH --output logs/weak_scaling_%A_%a.out

# Resources - reserve full node to avoid interference
#SBATCH --nodes 1
#SBATCH --ntasks 1
#SBATCH --cpus-per-task 48
#SBATCH --mem 8G

# Node specificities
#SBATCH --partition cpu

# Array: each task ID is the number of threads to use
#SBATCH --array 1,2,4,8,16,24,32,48

# Clean environment
#SBATCH --export NONE

# =============================================================================
# Job execution
# =============================================================================

# Set thread count from array task ID
export OMP_NUM_THREADS=${{SLURM_ARRAY_TASK_ID}}

# Base size for 1 thread
BASE_SIZE={WEAK_SCALING_BASE}

# Artificial serial delay (ms)
SERIAL_MS={ARTIFICIAL_SERIAL_MS}

# Scale dimensions by sqrt(N) to keep work per thread constant
# Using bc for floating point math, then converting to integer
SCALE=$(echo "scale=6; sqrt(${{SLURM_ARRAY_TASK_ID}})" | bc)
WIDTH=$(echo "${{BASE_SIZE}} * ${{SCALE}}" | bc | cut -d'.' -f1)
HEIGHT=${{WIDTH}}  # Keep square

# Output file
OUTPUT_FILE="output/weak_${{SLURM_ARRAY_TASK_ID}}cores.bin"

echo "=== Weak Scaling Experiment ==="
echo "Job ID: ${{SLURM_JOB_ID}}"
echo "Array Task ID: ${{SLURM_ARRAY_TASK_ID}}"
echo "Threads: ${{OMP_NUM_THREADS}}"
echo "Scale factor: ${{SCALE}}"
echo "Problem size: ${{WIDTH}} x ${{HEIGHT}}"
echo "Artificial serial delay: ${{SERIAL_MS}} ms"
echo ""

# Run the Julia set computation
./julia_set ${{WIDTH}} ${{HEIGHT}} ${{OUTPUT_FILE}} ${{SERIAL_MS}}
'''

with open('job_weak_scaling.sh', 'w') as f:
    f.write(weak_scaling_script)

print("Generated job_weak_scaling.sh")
print(f"  - Base size: {WEAK_SCALING_BASE}x{WEAK_SCALING_BASE}")
print(f"  - Artificial serial delay: {ARTIFICIAL_SERIAL_MS} ms")

### Submit the Weak Scaling Job

Run this command in your terminal:

```bash
sbatch job_weak_scaling.sh
```

Monitor your jobs with:
```bash
squeue -u $USER
```

---

## 9. Results Analysis

Once all jobs have completed, run the following cells to parse the output files and generate plots.

In [ ]:
import os
import re
import glob
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def parse_slurm_output(filename):
    """Parse a SLURM output file to extract timing information."""
    result = {}
    with open(filename, 'r') as f:
        content = f.read()
        
        # Extract key-value pairs
        for line in content.split('\n'):
            if ':' in line:
                parts = line.split(':', 1)
                if len(parts) == 2:
                    key = parts[0].strip()
                    value = parts[1].strip()
                    
                    # Try to convert to number
                    try:
                        if '.' in value:
                            result[key] = float(value)
                        else:
                            result[key] = int(value)
                    except ValueError:
                        result[key] = value
    
    return result

def collect_results(pattern):
    """Collect results from all matching SLURM output files."""
    files = glob.glob(pattern)
    results = []
    
    for f in files:
        data = parse_slurm_output(f)
        if 'Threads' in data and 'Time' in data:
            results.append(data)
    
    # Sort by thread count
    results.sort(key=lambda x: x['Threads'])
    return results

# Amdahl's law function for curve fitting
def amdahl_speedup(N, p):
    """Amdahl's Law: S(N) = 1 / ((1-p) + p/N)"""
    return 1.0 / ((1.0 - p) + p / N)

print("Analysis functions defined.")

In [ ]:
# Collect strong scaling results
strong_results = collect_results('logs/strong_scaling_*.out')

print("Strong Scaling Results:")
print("-" * 50)
print(f"{'Threads':>10} {'Time (s)':>12} {'Speedup':>10} {'Efficiency':>12}")
print("-" * 50)

if strong_results:
    t1 = strong_results[0]['Time']  # Baseline (1 thread)
    
    for r in strong_results:
        threads = r['Threads']
        time = r['Time']
        speedup = t1 / time
        efficiency = speedup / threads
        print(f"{threads:>10} {time:>12.4f} {speedup:>10.2f} {efficiency:>12.2f}")
else:
    print("No results found. Have the jobs completed?")
    print("Check with: squeue -u $USER")

In [ ]:
# Collect weak scaling results
weak_results = collect_results('logs/weak_scaling_*.out')

print("Weak Scaling Results:")
print("-" * 60)
print(f"{'Threads':>10} {'Size':>12} {'Time (s)':>12} {'Scaled Speedup':>15}")
print("-" * 60)

if weak_results:
    t1 = weak_results[0]['Time']  # Baseline (1 thread)
    
    for r in weak_results:
        threads = r['Threads']
        width = r.get('Width', '?')
        height = r.get('Height', '?')
        time = r['Time']
        # For weak scaling, "scaled speedup" = N * T1 / TN
        # (how much more work we did in the same time)
        scaled_speedup = threads * t1 / time
        print(f"{threads:>10} {width}x{height:>6} {time:>12.4f} {scaled_speedup:>15.2f}")
else:
    print("No results found. Have the jobs completed?")
    print("Check with: squeue -u $USER")

---

## 10. Scaling Plots

In [ ]:
def plot_strong_scaling(results):
    """Plot strong scaling results with fitted Amdahl's Law curve."""
    if not results:
        print("No strong scaling results to plot.")
        return
    
    threads = np.array([r['Threads'] for r in results])
    times = np.array([r['Time'] for r in results])
    t1 = times[0]
    speedup = t1 / times
    efficiency = speedup / threads
    
    # Fit Amdahl's Law to the data
    try:
        popt, pcov = curve_fit(amdahl_speedup, threads, speedup, p0=[0.9], bounds=(0.5, 0.9999))
        p_estimated = popt[0]
        p_std = np.sqrt(pcov[0, 0])
    except:
        # Fallback to simple estimation
        S_max = speedup[-1]
        N_max = threads[-1]
        p_estimated = (1 - 1/S_max) / (1 - 1/N_max) if N_max > 1 else 0.95
        p_estimated = min(0.999, max(0.5, p_estimated))
        p_std = 0
    
    # Theoretical curves
    n_theory = np.linspace(1, max(threads), 100)
    ideal = n_theory
    amdahl_fit = amdahl_speedup(n_theory, p_estimated)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Speedup plot
    ax1 = axes[0]
    ax1.plot(n_theory, ideal, 'k--', label='Ideal (linear)', linewidth=1.5)
    ax1.plot(n_theory, amdahl_fit, 'b-', label=f"Amdahl fit (p={p_estimated:.4f})", linewidth=2)
    ax1.plot(threads, speedup, 'ro', label='Measured', markersize=10)
    ax1.set_xlabel('Number of Threads', fontsize=12)
    ax1.set_ylabel('Speedup', fontsize=12)
    ax1.set_title('Strong Scaling: Speedup', fontsize=14)
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(0, max(threads) + 2)
    ax1.set_ylim(0, max(max(speedup), amdahl_fit[-1]) * 1.1)
    
    # Efficiency plot
    ax2 = axes[1]
    ax2.axhline(y=1.0, color='k', linestyle='--', label='Ideal (100%)', linewidth=1.5)
    ax2.plot(n_theory, amdahl_fit / n_theory, 'b-', label=f"Amdahl fit (p={p_estimated:.4f})", linewidth=2)
    ax2.plot(threads, efficiency, 'ro', label='Measured', markersize=10)
    ax2.set_xlabel('Number of Threads', fontsize=12)
    ax2.set_ylabel('Efficiency (Speedup / Threads)', fontsize=12)
    ax2.set_title('Strong Scaling: Efficiency', fontsize=14)
    ax2.legend(fontsize=10)
    ax2.grid(True, alpha=0.3)
    ax2.set_xlim(0, max(threads) + 2)
    ax2.set_ylim(0, 1.1)
    
    plt.tight_layout()
    plt.savefig('output/strong_scaling_plot.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\nFitted parallel fraction: p = {p_estimated:.4f}" + (f" ± {p_std:.4f}" if p_std > 0 else ""))
    print(f"Serial fraction: (1-p) = {1-p_estimated:.4f}")
    print(f"Theoretical max speedup (Amdahl): {1/(1-p_estimated):.2f}x")

plot_strong_scaling(strong_results)

In [ ]:
def plot_weak_scaling(results):
    """Plot weak scaling results."""
    if not results:
        print("No weak scaling results to plot.")
        return
    
    threads = np.array([r['Threads'] for r in results])
    times = np.array([r['Time'] for r in results])
    t1 = times[0]
    
    # For weak scaling, we want time to stay constant (ideal)
    # Scaled speedup = N * T1 / TN (how much more work in same time)
    scaled_speedup = threads * t1 / times
    
    # Weak scaling efficiency
    weak_efficiency = t1 / times
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Execution time plot
    ax1 = axes[0]
    ax1.axhline(y=t1, color='k', linestyle='--', label=f'Ideal ({t1:.2f}s)', linewidth=1.5)
    ax1.plot(threads, times, 'go', label='Measured', markersize=10)
    # Fit a line to show trend
    z = np.polyfit(threads, times, 1)
    p_line = np.poly1d(z)
    ax1.plot(threads, p_line(threads), 'g-', linewidth=2, alpha=0.7)
    ax1.set_xlabel('Number of Threads', fontsize=12)
    ax1.set_ylabel('Execution Time (s)', fontsize=12)
    ax1.set_title('Weak Scaling: Execution Time', fontsize=14)
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(0, max(threads) + 2)
    
    # Scaled speedup plot
    ax2 = axes[1]
    ax2.plot(threads, threads, 'k--', label='Ideal (linear)', linewidth=1.5)
    ax2.plot(threads, scaled_speedup, 'go', label='Measured', markersize=10)
    # Fit a line through origin to show actual scaling
    slope = np.sum(threads * scaled_speedup) / np.sum(threads**2)
    ax2.plot(threads, slope * threads, 'g-', linewidth=2, alpha=0.7, label=f'Fit (slope={slope:.3f})')
    ax2.set_xlabel('Number of Threads', fontsize=12)
    ax2.set_ylabel('Scaled Speedup (N × T₁ / Tₙ)', fontsize=12)
    ax2.set_title('Weak Scaling: Scaled Speedup', fontsize=14)
    ax2.legend(fontsize=10)
    ax2.grid(True, alpha=0.3)
    ax2.set_xlim(0, max(threads) + 2)
    ax2.set_ylim(0, max(threads) * 1.1)
    
    plt.tight_layout()
    plt.savefig('output/weak_scaling_plot.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Calculate weak scaling efficiency
    print(f"\nWeak Scaling Efficiency (T1/TN):")
    for n, eff in zip(threads, weak_efficiency):
        print(f"  {n:2d} threads: {eff:.2f}")

plot_weak_scaling(weak_results)

---

## 11. Julia Set Visualization

Let's visualize one of the computed Julia set images:

In [ ]:
# Display the largest computed image (48 cores, strong scaling)
image_file = 'output/strong_48cores.bin'

if os.path.exists(image_file):
    img, w, h = read_julia_image(image_file)
    
    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(img, cmap='hot')
    plt.colorbar(im, ax=ax, label='Iterations', shrink=0.8)
    ax.set_title(f'Julia Set (C = -0.8 + 0.156i)\n{w} × {h} pixels', fontsize=14)
    ax.axis('off')
    plt.savefig('output/julia_set_visualization.png', dpi=150, bbox_inches='tight')
    plt.show()
else:
    print(f"Image file not found: {image_file}")
    print("Run the strong scaling experiment first.")

### Weak Scaling: Resolution Comparison

In weak scaling, higher thread counts compute larger images. To visualize the difference in resolution, we crop the **same physical region** (the center of the fractal) from each image. With more threads/pixels, we see finer details in the fractal structure.

In [ ]:
# Compare images from weak scaling by showing the same region
weak_files = sorted(glob.glob('output/weak_*cores.bin'), 
                    key=lambda x: int(re.search(r'weak_(\d+)cores', x).group(1)))

if len(weak_files) >= 4:
    # Get the smallest image dimensions (1 core) for the crop size
    img_1core, w_base, h_base = read_julia_image(weak_files[0])
    crop_size = min(w_base, h_base)  # This is the base resolution (e.g., 1000)
    
    # Select representative images
    selected_indices = [0, 2, 4, -1]  # 1, 4, 16, 48 cores
    selected = [weak_files[i] for i in selected_indices if i < len(weak_files)]
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    
    for ax, f in zip(axes.flat, selected):
        if os.path.exists(f):
            img, w, h = read_julia_image(f)
            cores = int(re.search(r'weak_(\d+)cores', f).group(1))
            
            # Crop the center region (same physical area in fractal coordinates)
            # The base image shows the full [-1.5, 1.5] x [-1.5, 1.5] domain
            # For larger images, we take the center crop_size x crop_size pixels
            # which corresponds to a smaller region in fractal space
            center_y, center_x = h // 2, w // 2
            half_crop = crop_size // 2
            crop = img[center_y - half_crop:center_y + half_crop,
                       center_x - half_crop:center_x + half_crop]
            
            ax.imshow(crop, cmap='hot')
            ax.set_title(f'{cores} cores: {w}×{h} total\n(showing {crop_size}×{crop_size} center crop)', fontsize=11)
            ax.axis('off')
    
    plt.suptitle('Weak Scaling: Same Region, Increasing Resolution\n'
                 'More cores → larger image → finer detail in center crop', 
                 fontsize=13, y=1.02)
    plt.tight_layout()
    plt.savefig('output/weak_scaling_images.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\nNote: All subplots show a {crop_size}×{crop_size} pixel crop from the center.")
    print("Larger simulations (more cores) resolve finer fractal details.")
else:
    print("Not enough weak scaling images found.")
    print("Run the weak scaling experiment first.")

---

## 12. Discussion Questions

After completing the experiments, consider these questions:

1. **Strong Scaling:**
   - Does the measured speedup match Amdahl's Law prediction?
   - What is the fitted parallel fraction of the code?
   - At what thread count does adding more cores become inefficient (efficiency < 50%)?
   - Try re-running with `ARTIFICIAL_SERIAL_MS = 0`. How does the fitted `p` change?

2. **Weak Scaling:**
   - Does execution time stay constant as we scale up?
   - Why might weak scaling efficiency decrease with more cores?
   - Which scaling behavior (strong or weak) is more relevant for your research?

3. **Practical Considerations:**
   - Why did we reserve the full node even when using fewer cores?
   - How would results differ if other jobs were running on the same node?
   - What factors besides serial code fraction affect scaling? (hint: memory bandwidth, cache effects)

---

**Congratulations!** You have successfully measured strong and weak scaling on an HPC cluster and connected the results to Amdahl's and Gustafson's Laws.